# L7: Build a Crew to Tailor Job Applications

In this lesson, you will built your first multi-agent system.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `mistral-small:24b`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, pretty_print_result
from dotenv import load_dotenv

load_dotenv('creds.env')  # Adjust path if needed


from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = "testapikey"

ollama_llm = ChatOpenAI(
model="ollama/mistral-small:24b",
base_url="http://localhost:11434"
)

## crewAI Tools

In [4]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./fake_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md')

/Users/skasmani/Downloads/personal/github/CrewAI-Worksatation/crewai-venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/Users/skasmani/Downloads/personal/github/CrewAI-Worksatation/crewai-venv/lib/python3.12/site-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator("website_url")
/Users/skasmani/Downloads/personal/github/C

- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [5]:
# from IPython.display import Markdown, display
# display(Markdown("./fake_resume.md"))

## Creating Agents

In [6]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    llm=ollama_llm,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [7]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    llm=ollama_llm,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [8]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    llm=ollama_llm,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [9]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    llm=ollama_llm,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Creating Tasks

In [10]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [11]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [12]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [13]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

In [14]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [15]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [ ]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: a1c106fc-56bd-4620-a009-81e8e6d54a19                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: e9ef94ba-8296-4895-a89c-8938d11deeab
       Status: Executing Task...

🚀 Crew: crew
├── 📋 Task: e9ef94ba-8296-4895-a89c-8938d11deeab
│      Status: Executing Task...
└── 📋 Task: 58e6eac9-fb17-4515-ab87-5464aec33416
       Status: Executing Task...

🚀 Crew: crew
├── 📋 Task: e9ef94ba-8296-4895-a89c-8938d11deeab
│      Status: Executing Task...
└── 📋 Task: 58e6eac9-fb17-4515-ab87-5464aec33416
       Status: Executing Task...
    └── 🤖 Agent: Tech Job Researcher
            Status: In Progress

# Agent: Tech Job Researcher
## Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.


...

🚀 Crew: crew
├── 📋 Task: e9ef94ba-8296-4895-a89c-8938d11deeab
│      Assigned to: Tech Job Researcher
│      Status: ✅ Completed
├── 📋 Task: 58e6eac9-fb17-4515-ab87-5464aec33416
│      Assigned to: Personal Profiler for Engineers
│      Status: ✅ Completed
│   ├── 🤖 Agent: Tech Job Researcher
│   │       Status: In Progress
│   │   └── 🧠 Thinking...
│   ├── 🤖 Agent: Personal Profiler for Engineers
│   │       Status: ✅ Completed
│   │   ├── 🔧 Used Read website content (3)
│   │   └── 🔧 Used Search the internet with Serper (2)
│   └── 🤖 Agent: Tech Job Researcher
│           Status: ✅ Completed
│       ├── 🔧 Used Read website content (7)
│       └── 🔧 Used Search the internet with Serper (4)
├── 📋 Task: 7fa9280c-f185-42da-95e8-891c7a157136
│      Assigned to: Resume Strategist for Engineers
│      Status: ✅ Completed
│   └── 🤖 Agent: Resume Strategist for Engineers
│           Status: ✅ Completed
│       ├── 🔧 Using Read a file's content (2)
│       ├── 🔧 Using Update the file's content (1)
│       ├── 🔧 Failed Search a MDX's content (3)
│       └── 🔧 Used Search the internet with Serper (6)
└── 📋 Task: a6203f18-fa67-449b-822b-3261fa42fed2
       Status: Executing Task...
    └── 🤖 Agent: Engineering Interview Preparer
            Status: In Progress
        ├── 🔧 Used Read a file's content (3)
        ├── 🔧 Using Search the web (1)
        └── 🔧 Used Search the internet with Serper (7)

🚀 Crew: crew
├── 📋 Task: e9ef94ba-8296-4895-a89c-8938d11deeab
│      Assigned to: Tech Job Researcher
│      Status: ✅ Completed
├── 📋 Task: 58e6eac9-fb17-4515-ab87-5464aec33416
│      Assigned to: Personal Profiler for Engineers
│      Status: ✅ Completed
│   ├── 🤖 Agent: Tech Job Researcher
│   │       Status: In Progress
│   │   └── 🧠 Thinking...
│   ├── 🤖 Agent: Personal Profiler for Engineers
│   │       Status: ✅ Completed
│   │   ├── 🔧 Used Read website content (3)
│   │   └── 🔧 Used Search the internet with Serper (2)
│   └── 🤖 Agent: Tech Job Researcher
│           Status: ✅ Completed
│       ├── 🔧 Used Read website content (7)
│       └── 🔧 Used Search the internet with Serper (4)
├── 📋 Task: 7fa9280c-f185-42da-95e8-891c7a157136
│      Assigned to: Resume Strategist for Engineers
│      Status: ✅ Completed
│   └── 🤖 Agent: Resume Strategist for Engineers
│           Status: ✅ Completed
│       ├── 🔧 Using Read a file's content (2)
│       ├── 🔧 Using Update the file's content (1)
│       ├── 🔧 Failed Search a MDX's content (3)
│       └── 🔧 Used Search the internet with Serper (6)
└── 📋 Task: a6203f18-fa67-449b-822b-3261fa42fed2
       Status: Executing Task...
    └── 🤖 Agent: Engineering Interview Preparer
            Status: In Progress
        ├── 🔧 Used Read a file's content (3)
        ├── 🔧 Using Search the web (1)
        ├── 🔧 Used Search the internet with Serper (7)
        └── 🧠 Thinking...

Final Answer: 
- Proven experience as a Full Stack Developer or similar role
- Experience developing desktop and mobile applications
- Familiarity with common stacks
- Knowledge of multiple front-end languages and libraries (e.g. HTML/ CSS, JavaScript, XML, jQuery)
- Knowledge of multiple object-oriented languages (e.g. Java, Python) and multiple frameworks (e.g. Angular, React, Node.js)
- Familiarity with databases (e.g. MySQL, MongoDB), web servers (e.g. Apache) and UI/UX design
- Excellent communication skills and the ability to have in-depth technical discussions with both the engineering team and business people
- Excellent English language skills
- Self-starter and comfort working in an early-stage environment
- Strong project management and organizational skills
- BSc in Computer Science, Mathematics or similar field; Master’s or PhD degree is a plus
- Understanding of AI/ML models
- Understanding of big data tools, including Hadoop, Spark, Kafka, etc.
- Proficiency in HTML, CSS

Final Answer:
# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Profile
Noah Williams is a distinguished Software Engineering Leader with an 18-year tenure in the technology industry, where he has excelled in leading both remote and in-office engineering teams. His expertise spans across software development, process innovation, and enhancing team collaboration. He is highly proficient in programming languages such as Ruby, Python, JavaScript, TypeScript, and Elixir, alongside deep expertise in various front end frameworks. Noah's significant experience in data science and machine learning has enabled him to spearhead successful deployments of scalable AI solutions and innovative data model development.

## Work History

### DataKernel: Director of Software Engineering (remote) — 2022 - Present
- Noah has transformed the engineering division into a key revenue pillar for DataKernel, rapidly expanding the customer base from inception to a robust community.

I need to generate interview questions and talking points based on Noah's resume and the job requirements.

Action: Read a file's content
Action Input: {}
 


# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Profile
Noah Williams is a distinguished Software Engineering Leader with an 18-year tenure in the technology industry, where he has excelled in leading both remote and in-office engineering teams. His expertise spans across software development, process innovation, and enhancing team collaboration. He is highly proficient in programming languages such as Ruby, Python, JavaScript, TypeScript, and Elixir, alongside deep expertise in various front end frameworks. Noah's significant experience in data science and machine learning has enabled him to spearhead successful deployments of scalable AI solutions and innovative data model development.

## Work History

### DataKernel: Director of Software Engineering (remote) — 2022 - Present
- Noah has transfo

- Dislplay the generated `tailored_resume.md` file.

In [ ]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

- Dislplay the generated `interview_materials.md` file.

In [18]:
display(Markdown("./interview_materials.md"))

Based on Noah Williams' extensive experience and skills, here are some interview questions and talking points that can be used to assess his fit for the Full Stack Developer role:

1. Can you walk us through a project where you successfully led a team in developing both desktop and mobile applications? What were the key challenges you faced, and how did you overcome them?
   
2. How do you stay updated with the latest trends and advancements in technology, particularly in terms of common stacks and front-end languages/libraries?
   
3. Could you provide an example of a time when you utilized multiple object-oriented languages and frameworks in a project? What was the outcome of implementing this approach?
   
4. In what ways have you demonstrated your proficiency in databases, web servers, and UI/UX design in previous projects?
   
5. How do you approach technical discussions with engineering teams and business stakeholders to ensure effective communication and understanding?
   
6. Can you share a project where you showcased your self-starting attitude and comfort working in an early-stage environment? What were the key contributions you made to the project's success?
   
7. With your background in AI/ML models and big data tools, how have you incorporated these technologies into your past projects to drive innovation and growth?
   
8. Describe your experience with HTML, CSS, and JavaScript in developing consumer applications and handling data. How do you prioritize data privacy regulations and best practices in your work?
   
9. Given your leadership roles in previous companies, how do you approach project management and organization to ensure successful outcomes?
   
10. How do you see your background in software engineering and data science aligning with the requirements of this Full Stack Developer role? What unique skills and experiences do you bring to the table that set you apart from other candidates?

These questions and talking points are designed to dive deep into Noah Williams' background, skills, and experiences to assess his suitability for the Full Stack Developer position based on the job requirements and his qualifications.

# CONGRATULATIONS!!!

## Share your accomplishment!
- Once you finish watching all the videos, you will see the "In progress" image on the bottom left turn into "Accomplished".
- Click on "Accomplished" to view the course completion page with your name on it.
- Take a screenshot and share on LinkedIn, X (Twitter), or Facebook.  
- **Tag @Joāo (Joe) Moura, @crewAI, and @DeepLearning.AI, (and a few of your friends if you'd like them to try out the course)**
- **Joāo and DeepLearning.AI will "like"/reshare/comment on your post!**

## Get a completion badge that you can add to your LinkedIn profile!
- Go to [learn.crewai.com](https://learn.crewai.com).
- Upload your screenshot of your course completion page.
- You'll get a badge from CrewAI that you can share!

(Joāo will also talk about this in the last video of the course.)